In [50]:
from sklearn import datasets
import matplotlib.pyplot as plt

features = [0, 9]

wine = datasets.load_wine()
data = wine.data[:, features]
targets = wine.target

plt.scatter(data[:, 0], data[:, 1], c=targets, s = 15, cmap = plt.cm.brg)
plt.xlabel(wine.feature_names[features[0]])
plt.ylabel(wine.feature_names[features[1]])

plt.show()

**Uma boa prática é normalizar nossos dados. No entanto podemos rodar ao final da aula para ver o impacto de *não* fazer isso**

In [43]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data = scaler.fit_transform(data)

plt.scatter(data[:, 0], data[:, 1], c = targets, s = 15, cmap=plt.cm.brg)
plt.xlabel(wine.feature_names[features[0]])
plt.ylabel(wine.feature_names[features[1]])

plt.show()

invalid command name "140110819239552process_stream_events"
    while executing
"140110819239552process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


### Instanciando nossa rede neural

In [51]:
import torch
from torch import nn

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

input_size = data.shape[1]
hidden_size = 32  # escolha do programador
out_size = len(wine.target_names) # numero de classes

net = nn.Sequential(
    nn.Linear(input_size, hidden_size), # Camada escondida
    nn.ReLU(), # Funcao de ativacao
    nn.Linear(hidden_size, out_size), # Camada linear
    nn.Softmax() # Transforma em uma distribuicao de probabilidades
)

net = net.to(device)
print(net)

Sequential(
  (0): Linear(in_features=2, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=3, bias=True)
  (3): Softmax(dim=None)
)


### Visualizando a fronteira de decisao

In [53]:
import numpy as np


def plot_boundary(X, y, model):
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1

    spacing = min(x_max - x_min, y_max - y_min) / 100

    XX, YY = np.meshgrid(
        np.arange(x_min, x_max, spacing), np.arange(y_min, y_max, spacing)
    )

    data = np.hstack((XX.ravel().reshape(-1, 1), YY.ravel().reshape(-1, 1)))

    # # Para problemas binarios
    # db_prob = model(Variable(torch.Tensor(data)).cuda())
    # clf = np.where(db_prob.cpu().data < 0.5, 0, 1)

    # Para problemas multiclasse
    db_prob = model(torch.Tensor(data).to(device))
    clf = np.argmax(db_prob.cpu().data.numpy(), axis=-1)

    Z = clf.reshape(XX.shape)

    plt.contourf(XX, YY, Z, cmap=plt.cm.brg, alpha=0.5)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors="k", s = 25, cmap=plt.cm.brg)

In [54]:
plot_boundary(data, targets, net)
plt.show()

In [55]:
from torch import optim

# Funcao de perda
criterion = nn.CrossEntropyLoss().to(device) # to(device) serve para usar a GPU, mas nao funciona em GPUs sem ser NVIDIA

# Otimizador: descida do gradiente
# Descida estocastica do gradiente
optimizer = optim.SGD(net.parameters(), lr=1e-3)

### Casting dos dados

In [56]:
X = torch.FloatTensor(data).to(device)
Y = torch.LongTensor(targets).to(device)

# Ao rodar, percebemos que apenas uma iteracao nao eh suficiente!
# Portanto, precisamos fazer um loop
for i in range(100):
    # Forward
    pred = net(X)
    loss = criterion(pred, Y)

    # Backpropagation
    loss.backward()
    optimizer.step()

    plot_boundary(data, targets, net)
    
    if i % 10 == 0:
        # plt.figure()
        plt.show()  
# Visualizacao ruim, melhor fazer no colab